In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import cv2 as cv

In [ ]:
import cv2 as cv
import numpy as np
import glob
from matplotlib import pyplot as plt
list1=glob.glob('/content/drive/MyDrive/datatstes/fresh_banana/*.png')
list2=glob.glob('/content/drive/MyDrive/datatstes/rotten_banana/*.png')

In [ ]:
def labeling(name_list,label, X, y):
    for i in name_list:
        y.append(label)
        img=cv.imread(i,1) 
        img=np.asarray(cv.resize(img, (96, 96)))
        X.append(img)

X=[]
y=[]
labeling(list1,np.array([1, 0]),X,y)
labeling(list2,np.array([0, 1]),X,y)
X = np.asarray(X)
y = np.asarray(y)

In [ ]:
from sklearn.model_selection import train_test_split
img_train, img_test, labels_train, labels_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(img_train.shape, labels_train.shape)
print(img_test.shape, labels_test.shape)

(885, 96, 96, 3) (885, 2)
(222, 96, 96, 3) (222, 2)


In [ ]:
Path = '/content/drive/MyDrive/drive-download-20230323T162406Z-002/'

class_names = sorted(os.listdir(Path))
class_names.remove('labels.csv')
num_classes = len(class_names)

img_size = [96, 96, 3]
print(f'{num_classes} class: {class_names}\nImage size: {img_size}')

2 class: ['dog', 'other']
Image size: [96, 96, 3]


In [ ]:
images = []
labels = []
labels_df = pd.read_csv(Path + 'labels.csv')

print('Label Data Frame:')
labels_df.head()

Label Data Frame:


,Unnamed: 0,filename,label
0,0,QYjQz1LS7dATFSYYRfnOfk6ezX8Gp49589.jpg,other
1,1,rltnrLmjyQgZnuSpbTKjPrm24wjHL43876.jpg,other
2,2,LNZNJPLSMhZAo8siLJlLVdHoxCTU5T11617.jpg,other
3,3,5ghe8eSnGWzfpowvy0oeLnqkEPqYbO8045.jpg,other
4,4,2V3zbjyoxmG3zKDyxTpN2GpEvVolj811765.jpg,dog


In [ ]:
for image in labels_df.iloc:
    try:
      images.append(np.asarray(cv.resize(cv.imread(Path + image[2] + '/' + image[1], 1), img_size[0:2])[:, :, ::-1]))
      label = np.zeros(num_classes)
      label[class_names.index(image[2])] = 1
      labels.append(label)
    except:
      continue

images = np.asarray(images)
labels = np.asarray(labels)

print(f'\nlabels shape: {labels.shape}')
print(f'images shape: {images.shape}')

In [ ]:
from sklearn.model_selection import train_test_split
img_train, img_test, labels_train, labels_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)
print(img_train.shape, labels_train.shape)
print(img_test.shape, labels_test.shape)

(2425, 96, 96, 3) (2425, 2)
(607, 96, 96, 3) (607, 2)


In [ ]:
img_train_scaled = img_train / 255.0
img_test_scaled = img_test / 255.0

In [ ]:
def Conv2D(img, kernel):
  y, x = img.shape[0:2]
  a, b = kernel.shape
  
  y = y - a + 1
  x = x - b + 1
  
  new_img = np.zeros((y, x))
  
  for i in range(x):
    for j in range(y):
      new_img[j, i] = np.sum(img[j:j+a, i:i+b]*kernel)
  return new_img

In [ ]:
def Deconv2D(img, kernel):
    y, x = img.shape
    a, b = kernel.shape
    size_out = (a + y -1, b + x - 1)
    new_img = np.zeros(size_out)
    for i in range(y):
        for j in range(x):
            bit_matrix = kernel * img[i, j]
            new_img[i:i+kernel.shape[0], j:j+kernel.shape[1]] = new_img[i:i+kernel.shape[0], j:j+kernel.shape[1]] + bit_matrix
    return new_img


In [ ]:
class Conv2DLayer:
  def __init__(self, img_size_input, img_size_output):
    kernel_size = np.array(img_size_input[0:2]) - np.array(img_size_output[0:2]) + 1
    list_kernel = []
    for j in range(img_size_output[2]):
      kernel = []
      for i in range(img_size_input[2]):
        kernel.append(np.random.randn(kernel_size[0], kernel_size[1]))
      kernel = np.asarray(kernel).transpose((1, 2, 0))
      list_kernel.append(kernel)
    self.kernel_size = kernel_size
    list_kernel = np.asarray(list_kernel)
    self.list_kernel = list_kernel
    self.img_size_input = img_size_input
    self.img_size_output = img_size_output
    self.is_forward = False

  def ReLU(self, z):
    z[z < 0] = 0
    return z

  def convolution(self, img):
    z = []
    for j in self.list_kernel:
        temp_img = np.zeros(self.img_size_output[0:2])
        for i in range(self.img_size_input[2]):
          temp_img = temp_img + Conv2D(img[:,:,i], j[:,:,i])
        z.append(temp_img)
    return np.asarray(z).transpose((1, 2, 0))

  def forward(self, arr_img):
    arr_img_conv = []
    for img in arr_img:
      arr_img_conv.append(self.convolution(img))
    a = self.ReLU(np.asarray(arr_img_conv))
    self.a = list(a)
    self.x = arr_img
    self.is_forward = True
    return a

  def back_propagation(self, x):
    k = self.a
    dkernel = []
    for i in range(self.img_size_output[2]):
      dkernel.append(np.zeros_like(self.list_kernel[i]))
    dkernel = np.asarray(dkernel)
    for t in range(self.x.shape[0]):
      for i in range(self.img_size_output[2]):
        for j in range(self.img_size_input[2]):
            dkernel[i,:,:,j] = dkernel[i,:,:,j] + Conv2D(self.x[t,:,:,j], x[t,:,:,i])
    dkernel = dkernel / (len(k) * dkernel.max())

    dx = []
    for a in k:
      dx_C = np.zeros(self.img_size_input)
      for t in range(self.img_size_output[2]):
        for j in self.list_kernel:
          for i in range(self.img_size_input[2]):
            dx_C[:,:,i] = dx_C[:,:,i] + Deconv2D(a[:,:,t], j[:,:,i])
      dx.append(dx_C)
    dx = self.ReLU(np.asarray(dx))

    return np.asarray(dkernel), dx / (len(self.list_kernel) * dx.max())


In [ ]:
class MaxPooling2DLayer:
    def __init__(self, pool_size=(2, 2), strides=None):
        self.pool_size = pool_size
        if strides is None:
            self.strides = pool_size
        else:
            self.strides = strides
        self.is_forward = False

    def pooling(self, X, h_out, w_out, c, pool_height, pool_width, stride_height, stride_width):
        out = []
        for i in range(c):
          out_temp = np.zeros((h_out, w_out))
          for j in range(h_out):
            for k in range(w_out):
              out_temp[j, k] = X[j*stride_height:j*stride_height+pool_height, k*stride_width:k*stride_width+pool_width, i].max()
          out.append(out_temp)
        out = np.asarray(out)
        self.is_forward = True
        return out.transpose((1, 2, 0))

    def forward(self, X):
        n, h_in, w_in, c = X.shape
        pool_height, pool_width = self.pool_size
        stride_height, stride_width = self.strides
        h_out = (h_in - pool_height) // stride_height + 1
        w_out = (w_in - pool_width) // stride_width + 1
        out = []
        for i in range(n):
          out.append(self.pooling(X[i], h_out, w_out, c, pool_height, pool_width, stride_height, stride_width))
        out = np.asarray(out)
        self.x = X
        self.cache = [pool_height, pool_width, stride_height, stride_width]
        self.is_forward = True
        return out

    def back_propagation(self,dout):
        x = self.x
        n, h_back, w_back, c= x.shape
        pool_height, pool_width, stride_height, stride_width = self.cache
        
        dx = []
        for r in range(n):
          dx_C = []
          for k in range(c):
            dback = np.zeros((h_back, w_back))
            for i in range(dout[r,:,:,k].shape[0]):
              for j in range(dout[r,:,:,k].shape[1]):
                dback[i*stride_height:i*stride_height+pool_height, j*stride_width:j*stride_width+pool_width][np.unravel_index(np.argmax(x[r, i*stride_height:i*stride_height+pool_height, j*stride_width:j*stride_width+pool_width, k]), x[r, i*stride_height:i*stride_height+pool_height, j*stride_width:j*stride_width+pool_width, k].shape)] = dout[r,i,j,k]
            dx_C.append(dback)
          dx_C = np.asarray(dx_C)
          dx.append(dx_C)
        dx = np.asarray(dx)
        return dx.transpose((0, 2, 3, 1))

In [ ]:
class AvgPooling2DLayer(MaxPooling2DLayer):
    def pooling(self, X, h_out, w_out, c, pool_height, pool_width, stride_height, stride_width):
          out = []
          for i in range(c):
            out_temp = np.zeros((h_out, w_out))
            for j in range(h_out):
              for k in range(w_out):
                out_temp[j, k] = X[j*stride_height:j*stride_height+pool_height, k*stride_width:k*stride_width+pool_width, i].mean()
            out.append(out_temp)
          out = np.asarray(out)
          return out.transpose((1, 2, 0))

    def back_propagation(self,dout):
        x = self.x
        n, h_back, w_back, c= x.shape
        pool_height, pool_width, stride_height, stride_width = self.cache
        
        dx = []
        for r in range(n):
          dx_C = []
          for k in range(c):
            dback = np.zeros((h_back, w_back))
            for i in range(dout[r,:,:,k].shape[0]):
              for j in range(dout[r,:,:,k].shape[1]):
                dback[i*stride_height:i*stride_height+pool_height, j*stride_width:j*stride_width+pool_width] = dout[r,i,j,k] / (pool_height * pool_width)
            dx_C.append(dback)
          dx_C = np.asarray(dx_C)
          dx.append(dx_C)
        dx = np.asarray(dx)
        return dx.transpose((0, 2, 3, 1))
    

In [ ]:
class FlattenLayer:
  def __init__(self, input_shape):
    self.input_shape = input_shape
    self.is_forward = False

  def forward(self, data):
    out = []
    n = data.shape[0]
    self.n = n
    for i in range(n):
      out.append(data[i,:,:,:].transpose(2, 0, 1).ravel())
    self.is_forward = True
    return np.asarray(out / np.max(out))
  
  def back_propagation(self, dx):
    dback = []
    for i in range(self.n):
      dback.append(dx[i].reshape(self.input_shape[2],self.input_shape[0],self.input_shape[1]).transpose(1, 2, 0))
    return np.asarray(dback)

In [ ]:
class SigmoidLayer:
    def __init__(self, n_inp, n_out, bias=True):
        if bias: 
            n_inp += 1
            w = np.random.randn(n_inp, n_out)
            self.w = w
            self.bias = bias
            self.is_forward = False

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forward(self, X):
        if self.bias:
            X = np.hstack((np.ones((X.shape[0], 1)), X))

        z = np.dot(X, self.w)
        a = self.sigmoid(z)

        self.is_forward = True
        self.x = X
        self.a = a
        return a

    def back_propagation(self, delta):
        a = self.a
        dw = np.dot(self.x.T, delta * a * (1.0 - a))

        if self.bias:
            da = np.dot(delta * a * (1.0 - a), self.w.T)[:, 1:]
        else:
            da = np.dot(delta * a * (1.0 - a), self.w.T)

        return dw, da

In [ ]:
class SoftmaxLayer:
    def __init__(self, n_inp, n_out, bias=True):
        if bias: 
            n_inp += 1
        w = np.random.randn(n_inp, n_out)
        self.w = w
        self.bias = bias
        self.is_forward = False

    def softmax(self, z):
        return np.exp(z)/np.sum(np.exp(z), axis=1, keepdims=True)

    def forward(self, X):
        if self.bias:
            X = np.hstack((np.ones((X.shape[0], 1)), X))

        z = np.dot(X, self.w)
        a = self.softmax(z)

        self.is_forward = True
        self.x = X
        self.a = a
        return a
    
    def loss(self, y):
        y_pred = self.a
        return -np.mean(np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred)))

    def back_propagation(self, delta):
        a = self.a
        dw = np.dot(self.x.T, delta * a * (1.0 - a))

        if self.bias:
            da = np.dot(delta * a * (1.0 - a), self.w.T)[:, 1:]
        else:
            da = np.dot(delta * a * (1.0 - a), self.w.T)

        return dw, da

In [ ]:
block1 = [Conv2DLayer([96, 96, 3], [94, 94, 2]), 
          Conv2DLayer([94, 94, 2], [88, 88, 1]), 
          MaxPooling2DLayer()]
block2 = [Conv2DLayer([44, 44, 1], [40, 40, 1]), 
          Conv2DLayer([40, 40, 1], [32, 32, 1]), 
          AvgPooling2DLayer(), 
          FlattenLayer([16, 16, 1])]
block3 = [SigmoidLayer(16*16*1, 256, bias = True),
          SigmoidLayer(256, 128),
          SigmoidLayer(128, 64),
          SigmoidLayer(64, 32),
          SoftmaxLayer(32, 2)]

In [ ]:
model = [block1, block2, block3]
#luu lai ket qua tai day

In [ ]:
def predict(X, model):
  for i in model:
    for j in i:
      X = j.forward(X)
  return X

In [ ]:
def train_step(X, y, model, lr, alpha):
    y_pred = predict(X, model)
    delta = y_pred - y

    for i in reversed(block3):
        dw, delta = i.back_propagation(delta)
        i.w -= lr * dw

    for i in reversed(block2[-2:]):
        delta = i.back_propagation(delta)
    for i in reversed(block2[:-2]):
        dkernel, delta = i.back_propagation(delta)
        i.list_kernel -= lr * dkernel
    for i in reversed(block1[-1:]):
        delta = i.back_propagation(delta)
    for i in reversed(block1[:-1]):
        dkernel, delta = i.back_propagation(delta)
        i.list_kernel -= lr * dkernel + alpha * i.list_kernel
    
    print(f'w: {model[-1][-1].w}, loss: {model[-1][-1].loss(y)}')
  

In [ ]:
def batch_generator(X, y, batch_size=32):
    idx = np.arange(X.shape[0])
    np.random.shuffle(idx)
    n_batch = len(idx) // batch_size

    for i in range(n_batch + 1):
        i_start = i * batch_size
        i_stop = min((i+1) * batch_size, len(idx))
        idx_batch = idx[i_start: i_stop]
        X_batch = X[idx_batch, :]
        y_batch = y[idx_batch, :]
        yield X_batch, y_batch

In [ ]:
def train(X, y, model, lr, alpha, n_epochs):
    for i in range(n_epochs):
        print(f'epoch: {i}')
        for Xb, yb in batch_generator(X, y):
            train_step(Xb, yb, model, lr, alpha)

In [ ]:
train(img_train_scaled, labels_train, model, 0.01, 0.7, 1000)

epoch: 0
w: [[-2.52017412e-01  9.19822153e-01]
 [-1.13657130e+00 -2.35866416e-03]
 [ 1.01310874e-01  2.55511020e-01]
 [-9.69960671e-01 -1.90538444e+00]
 [-8.35729144e-01 -4.48679642e-01]
 [ 5.09296298e-01  2.21186743e-01]
 [ 2.06765969e+00  7.17304060e-01]
 [-1.68895247e-01 -3.53863873e-01]
 [-1.45553563e+00 -5.45759564e-01]
 [ 6.65536601e-01 -1.31651219e+00]
 [ 5.43585309e-01 -6.86272401e-01]
 [-1.20533718e+00 -8.42633521e-01]
 [-3.67735993e-01 -1.23514046e+00]
 [ 1.56058081e+00 -1.35369517e+00]
 [ 1.68120799e+00 -7.00717404e-01]
 [-1.07125943e+00  2.01122446e+00]
 [ 9.74121998e-02 -6.07880129e-01]
 [-7.35223899e-01 -2.59375097e-01]
 [ 8.06735463e-01  1.44820745e+00]
 [ 1.23272397e+00  5.57013193e-01]
 [ 1.48662910e+00 -1.21859324e-01]
 [ 1.02066908e+00  2.63739583e+00]
 [-8.78220654e-02 -4.24410974e-02]
 [-2.01378899e+00  1.95812327e+00]
 [ 2.36037645e-02  9.32446590e-01]
 [-5.16548780e-01  1.66760142e+00]
 [ 1.14117050e+00  8.53578800e-01]
 [ 1.34645511e+00 -6.69467792e-02]
 [-1.077

<ipython-input-216-25765f685e0e>:13: RuntimeWarning: invalid value encountered in true_divide
  return np.asarray(out / np.max(out))
<ipython-input-213-734e924f9849>:63: RuntimeWarning: invalid value encountered in true_divide
  return np.asarray(dkernel), dx / (len(self.list_kernel) * dx.max())


w: [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]], loss: nan
w: [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]], loss: nan
w: [[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]